In [1]:
from policy_net.environment import Environment
from policy_net.learning_agent import LearningAgent
from policy_net.animate import animate_species_ownership_with_static_layers, plot_agent_metrics, print_variable_significance, plot_dual_axis
from policy_net.game import play_game, finish_episode, log_channel_sums, save_checkpoint, load_checkpoint
from policy_net.policy_net import PolicyNet
from nca.constsants import CHANNELS, H, W, NUM_CHANNELS
from nca.nca_model import NCA, get_species_features_tensor,build_channel_mapping_from_species_list
from nca.generate_map import generate_training_world
import torch
import random
from nca.constsants import PLANT_RULES
from collections import Counter, defaultdict

16NUMCHANNELS


In [5]:
import pickle

from policy_net.supervised import generate_supervised_dataset

dataset = generate_supervised_dataset(num_samples=4000, trials_per_species=15)
with open("supervised_plant_dataset.pkl", "wb") as f:
    pickle.dump(dataset, f)

Sample 1/4000 complete.
Sample 2/4000 complete.
Sample 3/4000 complete.
Sample 4/4000 complete.
Sample 5/4000 complete.
Sample 6/4000 complete.
Sample 7/4000 complete.
Sample 8/4000 complete.
Sample 9/4000 complete.
Sample 10/4000 complete.
Sample 11/4000 complete.
Sample 12/4000 complete.
Sample 13/4000 complete.
Sample 14/4000 complete.
Sample 15/4000 complete.
Sample 16/4000 complete.
Sample 17/4000 complete.
Sample 18/4000 complete.
Sample 19/4000 complete.
Sample 20/4000 complete.
Sample 21/4000 complete.
Sample 22/4000 complete.
Sample 23/4000 complete.
Sample 24/4000 complete.
Sample 25/4000 complete.
Sample 26/4000 complete.
Sample 27/4000 complete.
Sample 28/4000 complete.
Sample 29/4000 complete.
Sample 30/4000 complete.
Sample 31/4000 complete.
Sample 32/4000 complete.
Sample 33/4000 complete.
Sample 34/4000 complete.
Sample 35/4000 complete.
Sample 36/4000 complete.
Sample 37/4000 complete.
Sample 38/4000 complete.
Sample 39/4000 complete.
Sample 40/4000 complete.
Sample 41

In [8]:
dataset

[]

In [6]:
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from policy_net.supervised import PlantPolicyDataset
from policy_net.game import play_game, finish_episode, log_channel_sums, save_checkpoint, load_checkpoint


def train_policy_net_supervised(policy_net, dataset, file_name="supervised_pn", batch_size=8, epochs=10, lr=1e-4):
    from torch.utils.data import DataLoader
    from torch.nn import CrossEntropyLoss
    import torch.optim as optim

    dataloader = DataLoader(PlantPolicyDataset(dataset), batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(policy_net.parameters(), lr=lr)
    species_loss_fn = CrossEntropyLoss()
    location_loss_fn = CrossEntropyLoss()

    policy_net.train()

    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            grid = batch["grid"].to("cuda")              # [B, 1, C+1, H, W]
            species_features = batch["species_features"].to("cuda")  # [B, S, F+1]
            target_species_id = batch["target_species_id"].to("cuda")
            target_location = batch["target_location"].to("cuda")

            grid = grid.squeeze(1)  # [B, C+1, H, W]

            species_logits, location_logits = policy_net(grid, species_features)

            loss_species = species_loss_fn(species_logits, target_species_id)
            loss_location = location_loss_fn(location_logits, target_location)
            loss = loss_species + loss_location

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

    save_checkpoint(policy_net, optimizer, 29000, f"{file_name}.pth")


In [7]:
import pickle

policy_net = PolicyNet(
    num_species=5,
    h=H,
    w=W
).to("cuda")

with open("supervised_plant_dataset.pkl", "rb") as f:
    dataset = pickle.load(f)

train_policy_net_supervised(policy_net, dataset, file_name="supervised_pn", batch_size=8, epochs=600)

Epoch 1/600, Loss: 4351.1749
Epoch 2/600, Loss: 4001.7606
Epoch 3/600, Loss: 3980.4706
Epoch 4/600, Loss: 3964.5946
Epoch 5/600, Loss: 3955.4855
Epoch 6/600, Loss: 3945.1393
Epoch 7/600, Loss: 3933.0283
Epoch 8/600, Loss: 3924.5544
Epoch 9/600, Loss: 3901.7074
Epoch 10/600, Loss: 3884.1637
Epoch 11/600, Loss: 3858.1419
Epoch 12/600, Loss: 3824.9451
Epoch 13/600, Loss: 3786.6234
Epoch 14/600, Loss: 3737.1789
Epoch 15/600, Loss: 3684.1168
Epoch 16/600, Loss: 3614.2640
Epoch 17/600, Loss: 3542.9979
Epoch 18/600, Loss: 3452.4581
Epoch 19/600, Loss: 3367.4366
Epoch 20/600, Loss: 3262.2824
Epoch 21/600, Loss: 3163.8238
Epoch 22/600, Loss: 3060.0190
Epoch 23/600, Loss: 2945.3479
Epoch 24/600, Loss: 2841.1276
Epoch 25/600, Loss: 2723.9861
Epoch 26/600, Loss: 2604.7243
Epoch 27/600, Loss: 2496.0366
Epoch 28/600, Loss: 2376.8569
Epoch 29/600, Loss: 2284.5681
Epoch 30/600, Loss: 2158.7570
Epoch 31/600, Loss: 2065.5086
Epoch 32/600, Loss: 1961.7238
Epoch 33/600, Loss: 1845.0616
Epoch 34/600, Loss: